In [1]:
#https://rapidapi.com/apidojo/api/yahoo-finance1?endpoint=apiendpoint_f6ed13cb-defc-468f-a6a0-ee4ad28bf8af
import requests
import json
def callApi(url, stock):

    querystring = {"symbol": stock }

    headers = {
        'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com",
        'x-rapidapi-key': "b057006049msh2dd6f48e0283787p151e0djsnf76cd322583c"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    balance_sheet = json.loads(response.text)
    return balance_sheet

In [2]:
def pegRatio(stocks):
    for stock in stocks:
        result = callApi("https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-summary", stock)
        print(stock, result["defaultKeyStatistics"]["pegRatio"], "\n")
        print(stock, result["defaultKeyStatistics"]["priceToBook"], "\n")
        

In [3]:
pegRatio(['FB'])

FB {'raw': 2.15, 'fmt': '2.15'} 

FB {'raw': 7.5777364, 'fmt': '7.58'} 



In [91]:
#https://github.com/petercerno/good-morning
url = 'https://financials.morningstar.com/ratios/r.html?&callback=?&t={t}&region={reg}&culture={cult}&cur={cur}'.format(t='AAPL', reg='GBR', cult='en_US', cur='USD')
#url = 'https://financials.morningstar.com/ajax/exportKR2CSV.html?&callback=?&t={t}&region={reg}&culture={cult}&cur={cur}'.format(t='AAPL', reg='GBR', cult='en_US', cur='USD')
#url = (r'http://financials.morningstar.com/ajax/exportKR2CSV.html?'+ r'&callback=?&t={t}&region={reg}&culture={cult}&cur={cur}'.format(t='AAPL', reg='USA', cult='en_US', cur='USD'))

In [92]:
url

'https://financials.morningstar.com/ratios/r.html?&callback=?&t=AAPL&region=GBR&culture=en_US&cur=USD'

In [93]:
import csv
import json
import numpy as np
import pandas as pd
import re
import urllib.request
from bs4 import BeautifulSoup
from datetime import date

In [94]:
def _parse_tables(response):
        u"""Parses the given csv response from financials.morningstar.com.

        :param response: Response from financials.morningstar.com.
        :return: List of pairs, where the first item is the name of the table
        (extracted from the response) and the second item is the corresponding
        pandas.DataFrame table containing the data.
        """
        # Regex pattern used to recognize csv lines containing financial data.
        num_commas = 5
        pat_commas = r'(.*,){%d,}' % num_commas
        # Resulting array of pairs (table_name, table_frame).
        tables = []
        table_name = None
        table_rows = None

        for line in response:
            line = line.decode(u'utf-8').strip()
            match = re.match(pat_commas, line)
            if match:
                for row in csv.reader([line]):
                    table_rows.append(row)
            else:
                if table_name and table_rows:
                    tables.append([table_name, pd.DataFrame(table_rows)])
                if line != u'':
                    table_name = line
                table_rows = []
        if table_name and table_rows:
            tables.append([table_name, pd.DataFrame(table_rows)])
        return tables


In [95]:
    def _parse_frames(tables, response_structure):
        u"""Returns an array of processed pandas.DataFrames based on the
        original list of tables and the special response_structure list.

        :param tables: Original list of tables (obtained from _parse_tables).
        :param response_structure: List of pairs (expected table name, new name
        assigned to the corresponding (processed) pandas.DataFrame).
        """

        #############################
        # Error Handling
        #############################

        # Fail Early on Empty String
        if len(tables) == 0:
            return ("MorningStar could not find the ticker")

        period_start = tables[0][1].ix[0][1]
        period_month = pd.datetime.strptime(period_start, u'%Y-%m').month
        #period_freq = pd.datetools.YearEnd(month=period_month)
        period_freq = pd.tseries.offsets.YearEnd(month=period_month)
        frames = []
        for index, (check_name, frame_name) in enumerate(response_structure):
            if frame_name and tables[index][0] == check_name:
                frame = KeyRatiosDownloader._process_frame(
                    tables[index][1], frame_name, period_start, period_freq)
                if frame is not None and frame.index.size > 0:
                    frames.append(frame)
        return frames

In [97]:
with urllib.request.urlopen(url) as response:
            tables = _parse_tables(response)
            response_structure = [
                # Original Name, New pandas.DataFrame Name
                (u'Financials', u'Key Financials'),
                (u'Key Ratios -> Profitability', u'Key Margins % of Sales'),
                (u'Key Ratios -> Profitability', u'Key Profitability'),
                (u'Key Ratios -> Growth', None),
                (u'Revenue %', u'Key Revenue %'),
                (u'Operating Income %', u'Key Operating Income %'),
                (u'Net Income %', u'Key Net Income %'),
                (u'EPS %', u'Key EPS %'),
                (u'Key Ratios -> Cash Flow', u'Key Cash Flow Ratios'),
                (u'Key Ratios -> Financial Health',
                 u'Key Balance Sheet Items (in %)'),
                (u'Key Ratios -> Financial Health',
                 u'Key Liquidity/Financial Health'),
                (u'Key Ratios -> Efficiency Ratios', u'Key Efficiency Ratios')]
            #frames = _parse_frames(tables, response_structure)
            print(tables)

[['"area":"reports.stocks",',                                  0                   1                     2  \
0  listpos:"leaderboard"+midPos1+"  reportsSponsorLink  button120x60topright   

             3             4       5                                 6 7  
0  chrtxt30top  button195x90  footer  tickertapebutton"+midPos2+extPos    ], ['}',                                                    0            1  \
0  </script><ul class='r_cnav_ads' ><li><a href="...  Morningstar   

                                                   2            3  \
0  width=750,height=600,top=30,left=30,toolbar=ye...  Morningstar   

                                                   4            5  \
0  width=500,height=255,top=30,left=30,toolbar=ye...  Morningstar   

                                                   6  
0  width=750,height=600,top=30,left=30,toolbar=ye...  ], ['</div>',                                                    0             1  \
0  <div class="f_Copyright_row1">Independ